### This script analysis data collected from patients in hospital. The paradigm was described in another word file.
Tasks last for 5 seconds.


Problem: The task start time is not specified. Solutioin: visually identify the starting point.

In [1]:
import numpy as np
from HaiMeiKang.config import *
import pandas
import mne

2024-07-23 10:53:52 - Start Program

pre_all: Running from CMD.
common_dl.py: Using CUDA.


In [2]:
folder=data_dir+'from_hospital/yexiansong_baseline/'

In [3]:
filelist=['rest-elbow flexion.xlsx'
,'rest-elbow extension.xlsx'
,'rest-wrist flexion.xlsx'
,'rest-wrist extension.xlsx'
,'rest-digit flexion.xlsx'
,'rest-digit extension.xlsx'
,'rest-pollicis flexion.xlsx'
,'rest-pollicis extesion.xlsx'
,'rest-thenar.xlsx'
,'rest-abductor.xlsx']

### Explore the data

In [4]:
f=filelist[0]
filename=folder+f
tmp=pandas.read_excel(filename)
data=tmp.to_numpy()[:,1:]
time=tmp.to_numpy()[:,0]
trigger=[1 if t%5==0 else 0 for t in time]

In [43]:
ch_names=['emg'+str(i) for i in range(data.shape[1])]+['stim']
ch_types=['eeg' for i in range(data.shape[1])]+['stim']
sf=2000
info = mne.create_info(ch_names=ch_names, sfreq=sf, ch_types=ch_types)
#tmp2=np.concatenate((data,np.asarray(trigger)[:,np.newaxis]))
raw = mne.io.RawArray(np.concatenate((data.transpose(),np.asarray(trigger)[np.newaxis,:]), axis=0), info)

In [45]:
raw.plot()

<MNEBrowseFigure size 2137x900 with 4 Axes>

![](img\1.png)

In [47]:
events = mne.find_events(raw, stim_channel="stim")

In [48]:
events

array([[ 10000,      0,      1],
       [ 20000,      0,      1],
       [ 30000,      0,      1],
       [ 40000,      0,      1],
       [ 50000,      0,      1],
       [ 60000,      0,      1],
       [ 70000,      0,      1],
       [ 80000,      0,      1],
       [ 90000,      0,      1],
       [100000,      0,      1],
       [110000,      0,      1],
       [120000,      0,      1],
       [130000,      0,      1],
       [140000,      0,      1],
       [150000,      0,      1],
       [160000,      0,      1],
       [170000,      0,      1],
       [180000,      0,      1],
       [190000,      0,      1],
       [200000,      0,      1],
       [210000,      0,      1],
       [220000,      0,      1],
       [230000,      0,      1],
       [240000,      0,      1]], dtype=int64)

In [54]:
#mapping={1,}
anas=mne.annotations_from_events(events,sfreq=sf)
raw.set_annotations(anas)

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,"10 EEG, 1 Stimulus"
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,2000.00 Hz
Highpass,0.00 Hz
Lowpass,1000.00 Hz


In [56]:
raw.plot()  # 动作延时很大，要么就是文件开头不是任务开始时刻

<MNEBrowseFigure size 2678x1193 with 4 Axes>

![](img\2.png)

### put them together

In [16]:
raws=[]
for i,f in enumerate(filelist):
    print('Reading '+f+'.')
    filename=folder+f
    tmp=pandas.read_excel(filename)
    data=tmp.to_numpy()[:,1:] # 11 columes, no time colume
    time=tmp.to_numpy()[:,0]
    trigger=[i+1 if t%7==0 else 0 for t in time]
    ch_names=['emg'+str(i) for i in range(data.shape[1])]+['stim']
    ch_types=['eeg' for i in range(data.shape[1])]+['stim']
    sf=2000
    info = mne.create_info(ch_names=ch_names, sfreq=sf, ch_types=ch_types)
    #tmp2=np.concatenate((data,np.asarray(trigger)[:,np.newaxis]))
    raw = mne.io.RawArray(np.concatenate((data.transpose(),np.asarray(trigger)[np.newaxis,:]), axis=0), info)
    events = mne.find_events(raw, stim_channel="stim")
    anas=mne.annotations_from_events(events,sfreq=sf)
    raw.set_annotations(anas)
    raws.append(raw)

Reading rest-elbow flexion.xlsx.
Reading rest-elbow extension.xlsx.
Reading rest-wrist flexion.xlsx.
Reading rest-wrist extension.xlsx.
Reading rest-digit flexion.xlsx.
Reading rest-digit extension.xlsx.
Reading rest-pollicis flexion.xlsx.
Reading rest-pollicis extesion.xlsx.
Reading rest-thenar.xlsx.
Reading rest-abductor.xlsx.


In [18]:
for i,raw in enumerate(raws):
    raw.save(data_dir+'from_hospital/'+str(i)+'.fif',overwrite=True)

In [24]:
mne.find_events(raws[0], stim_channel="stim") # events number: one is 25, others are all 24 events

array([[ 10000,      0,      1],
       [ 20000,      0,      1],
       [ 30000,      0,      1],
       [ 40000,      0,      1],
       [ 50000,      0,      1],
       [ 60000,      0,      1],
       [ 70000,      0,      1],
       [ 80000,      0,      1],
       [ 90000,      0,      1],
       [100000,      0,      1],
       [110000,      0,      1],
       [120000,      0,      1],
       [130000,      0,      1],
       [140000,      0,      1],
       [150000,      0,      1],
       [160000,      0,      1],
       [170000,      0,      1],
       [180000,      0,      1],
       [190000,      0,      1],
       [200000,      0,      1],
       [210000,      0,      1],
       [220000,      0,      1],
       [230000,      0,      1],
       [240000,      0,      1]], dtype=int64)

In [25]:
list_of_epochs=[]
for raw in raws:
    event=mne.find_events(raw, stim_channel="stim")
    epoch= mne.Epochs(raw, event, tmin=1.5, tmax=6.5, baseline=None)
    list_of_epochs.append(epoch.get_data())
        

In [36]:
trial_numbers=[list(range(epochi.shape[0])) for epochi in list_of_epochs]

test_trials=[random.sample(trial_number, 4) for trial_number in trial_numbers]
trial_numbers_left=[np.setdiff1d(trial_numbers[i],test_trials[i]).tolist() for i in range(len(trial_numbers))]
val_trials=[random.sample(trial_number, 4) for trial_number in trial_numbers_left]
train_trials=[np.setdiff1d(trial_numbers_left[i],val_trials[i]).tolist() for i in range(len(trial_numbers))]

In [40]:
test_epochs=[epochi[test_trials[clas],:,:] for clas,epochi in enumerate(list_of_epochs)]
val_epochs=[epochi[val_trials[clas],:,:] for clas,epochi in enumerate(list_of_epochs)]
train_epochs=[epochi[train_trials[clas],:,:] for clas,epochi in enumerate(list_of_epochs)]

In [42]:
len(test_epochs[0])

4

In [ ]:
from gesture.utils import windowed_data
gen_data_all=None
X_train,y_train,X_val,y_val,X_test,y_test=windowed_data(train_epochs,val_epochs,test_epochs,1000,500,
                                                        gen_data_all=gen_data_all,train_mode='original',method=None)

### Now wrap all above into another file: decoding_dl.py to loop the training iteration.